In [11]:
import os
import sys
sys.path.insert(0, '..')
from utils.dataset_processing import process_data

In [13]:
DATA_DIR = 'resources'
# Data dir = (1) working directory, (2) move out of test, (3) move out of src, and append to resources
DATA_DIR = os.path.join(os.getcwd(), '..', 'resources')
print("Data Directory:", DATA_DIR)

TRAIN_FILE = 'factkg_train_5k.pickle'
TEST_FILE = 'factkg_test_1k.pickle'
VALID_FILE = 'factkg_dev_300.pickle'

TRAIN_FILE_PATH = os.path.join(DATA_DIR, TRAIN_FILE)
TEST_FILE_PATH = os.path.join(DATA_DIR, TEST_FILE)
VALID_FILE_PATH = os.path.join(DATA_DIR, VALID_FILE)

Data Directory: d:\claimpkg\claimpkg-clone\src\test\..\resources


In [21]:
import pickle

train_data = None
test_data = None
valid_data = None
with open(TRAIN_FILE_PATH, 'rb') as f:
    train_data = pickle.load(f)
with open(TEST_FILE_PATH, 'rb') as f:
    test_data = pickle.load(f)
with open(VALID_FILE_PATH, 'rb') as f:
    valid_data = pickle.load(f)

In [22]:
train_updated_data, train_distinct_entities = process_data(train_data, remove_underscore=True)

test_updated_data, test_distinct_entities = process_data(test_data, remove_underscore=True)

valid_updated_data, valid_distinct_entities = process_data(valid_data, remove_underscore=True)

Processing data: 100%|██████████| 300/300 [00:00<00:00, 100031.10it/s]


In [ ]:
# Export the processed data back to pickle files
with open(os.path.join(DATA_DIR, 'factkg_train_5k_triplets.pickle'), 'wb') as f:
    pickle.dump(train_updated_data, f)

with open(os.path.join(DATA_DIR, 'factkg_test_1k_triplets.pickle'), 'wb') as f:
    pickle.dump(test_updated_data, f)

with open(os.path.join(DATA_DIR, 'factkg_dev_300_triplets.pickle'), 'wb') as f:
    pickle.dump(valid_updated_data, f)

In [24]:
all_join_entities = set()
all_join_entities.update(train_distinct_entities)
all_join_entities.update(test_distinct_entities)
all_join_entities.update(valid_distinct_entities)

print("Total distinct entities across all datasets:", len(all_join_entities))

Total distinct entities across all datasets: 5356


In [26]:
# Export as a pickle file contains a set, later use this for a trie reformation
with open(os.path.join(DATA_DIR, 'factkg_all_distinct_entities.pickle'), 'wb') as f:
    pickle.dump(all_join_entities, f)